In [1]:
# 📦 Imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

from transformers import AutoTokenizer, AutoModel
import torch
from tqdm.notebook import tqdm  # Better for Google Colab
import datetime
import os

# 📁 Chargement des données
df = pd.read_csv("full_data_fusionne.csv")

# 🧼 Prétraitement du texte
def clean_text(text):
    if pd.isnull(text):
        return ""
    return str(text).strip()

df['title'] = df['title'].apply(clean_text)
df['content'] = df['content'].apply(clean_text)
df['full_text'] = df['title'] + " " + df['content']

# 🕰️ Traitement de la date
df['published_at'] = pd.to_datetime(df['published_at'], errors='coerce')
df = df.dropna(subset=['published_at'])  # Supprimer les lignes sans date valide
df['timestamp'] = df['published_at'].astype(np.int64) // 10**9

# 🔧 Désactiver les warnings inutiles
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# 📌 Charger le modèle BERT arabe
model_name = "asafaya/bert-base-arabic"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)
model.eval()

# 🧾 Fonction d'embedding
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

# ⏳ Vectorisation des textes
bert_vectors = []
for i, text in enumerate(tqdm(df['full_text'].tolist(), desc="Vectorisation BERT", ncols=100)):
    try:
        emb = get_bert_embedding(text)
    except Exception as e:
        print(f"Erreur à la ligne {i}: {e}")
        emb = np.zeros(768)
    bert_vectors.append(emb)

bert_vectors = np.array(bert_vectors)

# 🧮 Encodage des autres colonnes
other_features = pd.DataFrame()

if 'tags' in df.columns:
    df['tags'] = df['tags'].fillna("")
    other_features['tags'] = df['tags']

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_features = encoder.fit_transform(other_features)

# 🎛️ Combinaison et pondération
scaler = MinMaxScaler()
date_scaled = scaler.fit_transform(df[['timestamp']])

bert_weight = 3.0
date_weight = 2.0
other_weight = 4.0

combined_features = np.hstack([
    bert_vectors * bert_weight,
    date_scaled * date_weight,
    encoded_features * other_weight
])

# 📆 Ajouter une colonne '3_days_group'
df['date_only'] = df['published_at'].dt.date
df = df.sort_values('published_at')

# Grouper les dates par période de 3 jours
start_date = df['date_only'].min()
df['days_since_start'] = (df['published_at'] - pd.Timestamp(start_date)).dt.days
df['group_3_days'] = (df['days_since_start'] // 3).astype(int)

# 🏷️ Clustering par groupe de 3 jours
final_clusters = np.full(len(df), -1)  # Initialiser avec -1

for group in tqdm(df['group_3_days'].unique(), desc="Clustering par tranche de 3 jours"):
    idx = df[df['group_3_days'] == group].index
    group_features = combined_features[idx]

    # Choisir un k dynamique selon le nombre d'articles dans ce groupe
    k = min(5, len(idx))  # au maximum 5 clusters, ou moins si peu d'articles

    if len(idx) >= 2:  # besoin d'au moins 2 éléments pour KMeans
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        group_labels = kmeans.fit_predict(group_features)
        final_clusters[idx] = group_labels
    else:
        final_clusters[idx] = 0  # Si un seul élément, cluster = 0

df['cluster'] = final_clusters

# ✅ Résultat final
print(df[['published_at', 'group_3_days', 'cluster']].head(20))
print(df['cluster'].value_counts().sort_index())
df.to_csv('/content/drive/MyDrive/projetSD/data_clusters.csv')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/334k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Vectorisation BERT:   0%|                                                 | 0/41554 [00:00<?, ?it/s]

Clustering par tranche de 3 jours:   0%|          | 0/122 [00:00<?, ?it/s]

      published_at  group_3_days  cluster
21604   2024-02-22             0        3
21494   2024-02-22             0        3
22027   2024-02-22             0        2
11021   2024-02-22             0        2
23909   2024-02-22             0        4
20556   2024-02-22             0        0
16593   2024-02-22             0        4
21499   2024-02-22             0        4
18852   2024-02-22             0        3
22493   2024-02-22             0        2
24921   2024-02-22             0        2
15413   2024-02-22             0        2
16555   2024-02-22             0        3
16482   2024-02-22             0        3
23560   2024-02-22             0        3
16418   2024-02-22             0        3
16425   2024-02-22             0        3
23803   2024-02-22             0        0
13675   2024-02-22             0        0
22537   2024-02-22             0        0
cluster
0    9715
1    9182
2    7918
3    7543
4    7196
Name: count, dtype: int64


In [2]:
df['clusters'] = df['group_3_days'] * 5 + df['cluster']

In [9]:
df['group_3_days'].value_counts() , df['clusters'].value_counts()

(group_3_days
 86     750
 116    679
 74     659
 111    652
 118    644
       ... 
 19     104
 15      99
 0       91
 10      89
 5       80
 Name: count, Length: 122, dtype: int64,
 clusters
 556    373
 590    350
 580    298
 570    276
 596    243
       ... 
 38       4
 98       2
 83       2
 552      1
 86       1
 Name: count, Length: 607, dtype: int64)

In [10]:
df[df['clusters']==86]

,source,title,url,published_at,tags,content,full_text,timestamp,date_only,days_since_start,group_3_days,cluster,clusters
21128,Sky_News_Arabic,إسرائيل تراجع تحذير الاحتماء في إشارة لنهاية ا...,https://www.skynewsarabia.com/middle-east/1706523,2024-04-14,أخبار إسرائيل,أطلقت إيران وابلا من الطائرات المسيرة المتفجرة...,إسرائيل تراجع تحذير الاحتماء في إشارة لنهاية ا...,1713052800,2024-04-14,52,17,1,86


In [11]:
df[df['group_3_days']==17]

,source,title,url,published_at,tags,content,full_text,timestamp,date_only,days_since_start,group_3_days,cluster,clusters
21131,Sky_News_Arabic,إيران تحذر من فتح أجواء أي دولة أمام هجمات إسر...,https://www.skynewsarabia.com/middle-east/1706488,2024-04-13,أخبار إيران,ذكرت وكالة مهر الإيرانية شبه الرسمية للأنباء أ...,إيران تحذر من فتح أجواء أي دولة أمام هجمات إسر...,1712966400,2024-04-13,51,17,4,89
25715,Sky_News_Arabic,حماس تسلم الوسطاء ردها على مقترح الهدنة,https://www.skynewsarabia.com/middle-east/1706466,2024-04-13,أخبار فلسطين,أعلنت حماس، يوم السبت، تسلميها الوسطاء في مصر ...,حماس تسلم الوسطاء ردها على مقترح الهدنة أعلنت ...,1712966400,2024-04-13,51,17,0,85
18954,Sky_News_Arabic,مصر تعبر عن قلقها إزاء التصعيد الإيراني الإسرا...,https://www.skynewsarabia.com/middle-east/1706489,2024-04-13,أخبار مصر,قالت وزارة الخارجية المصرية في بيان في وقت متأ...,مصر تعبر عن قلقها إزاء التصعيد الإيراني الإسرا...,1712966400,2024-04-13,51,17,4,89
20015,Sky_News_Arabic,إيران تبدأ هجومها على إسرائيل بعشرات المسيّرات...,https://www.skynewsarabia.com/middle-east/1706477,2024-04-13,أخبار إسرائيل,بدأت إيران هجوما على إسرائيل بعشرات الطائرات ا...,إيران تبدأ هجومها على إسرائيل بعشرات المسيّرات...,1712966400,2024-04-13,51,17,4,89
13766,Sky_News_Arabic,إيران تطلق أكثر من 200 طائرة مسيرة وصاروخ تجاه...,https://www.skynewsarabia.com/middle-east/1706497,2024-04-13,أخبار إسرائيل,قال الجيش الإسرائيلي إن إيران أطلقت أكثر من 20...,إيران تطلق أكثر من 200 طائرة مسيرة وصاروخ تجاه...,1712966400,2024-04-13,51,17,4,89
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23492,Sky_News_Arabic,الدولار يتمسك بمكاسبه والين يهبط مع تراجع رهان...,https://www.snabusiness.com/article/1706661,2024-04-15,عملات,استقر الدولار، خلال تعاملات الاثنين المبكرة، م...,الدولار يتمسك بمكاسبه والين يهبط مع تراجع رهان...,1713139200,2024-04-15,53,17,0,85
20503,Sky_News_Arabic,"""واشنطن بوست"": رد إسرائيل على إيران قد لا يكون...",https://www.skynewsarabia.com/middle-east/1706840,2024-04-15,أخبار إسرائيل,"كشف تقرير لصحيفة ""واشنطن بوست"" الأميركية، أن ا...","""واشنطن بوست"": رد إسرائيل على إيران قد لا يكون...",1713139200,2024-04-15,53,17,4,89
26359,Sky_News_Arabic,أي صدى لصواريخ إيران في غزة,https://www.skynewsarabia.com/program/newsroom...,2024-04-15,,في الوقت الذي تتصاعد فيه الجهود الدولية من أجل...,أي صدى لصواريخ إيران في غزة في الوقت الذي تتصا...,1713139200,2024-04-15,53,17,3,88
18566,Sky_News_Arabic,هجمات إيران.. الرد الإسرائيلي,https://www.skynewsarabia.com/video/1706682,2024-04-15,أخبار العالم,تعهدت إسرائيل بردّ كبير على الهجمات الإيرانية ...,هجمات إيران.. الرد الإسرائيلي تعهدت إسرائيل بر...,1713139200,2024-04-15,53,17,0,85
